In [1]:
from ARQMathCode.post_reader_record import DataReaderRecord
from scipy.stats import zscore
from sentence_transformers import SentencesDataset, SentenceTransformer, losses
import torch
from torch.utils.data import DataLoader

from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

In [2]:
clef_home_directory_file_path = '/home/xstefan3/arqmath/data/Collection'
dr = DataReaderRecord(clef_home_directory_file_path)

reading users
reading comments
reading votes
reading post links
reading posts


In [27]:
dr.post_parser.map_questions[1].answers[1].votes.__len__()

18

In [43]:
def upvotes_to_distance(all_q_votes):
    if len(all_q_votes) < 2:
        return [1.0] if all_q_votes[0] > 0 else [0.0]
    z_vals = zscore(all_q_votes)
#     z_vals_inv = (-1)*z_vals
    z_vals_norm = (z_vals - z_vals.min()) / (z_vals.max() - z_vals.min())
    return z_vals_norm


def examples_from_questions(questions):
    for q_i, q in questions.items():
        if q_i % 10000 == 0:
            print("Loading %s" % q_i)
        if q.answers is None:
            continue
        all_q_upvotes = [a.score for a in q.answers]
        all_q_dists = upvotes_to_distance(all_q_upvotes)

        for a_i, a in enumerate(q.answers):
            yield InputExample("%s_%s" % (q_i, a_i), [q.body, a.body], all_q_dists[a_i])


In [44]:
examples_len = len(dr.post_parser.map_questions)

train_dev_test_split = (int(0.05*examples_len), int(0.1*examples_len))

all_examples = list(examples_from_questions(dr.post_parser.map_questions))

Loading 20000
Loading 60000
Loading 110000
Loading 120000
Loading 140000
Loading 160000
Loading 190000
Loading 200000
Loading 280000
Loading 320000
Loading 370000
Loading 390000
Loading 430000
Loading 440000
Loading 450000
Loading 460000
Loading 470000
Loading 500000
Loading 570000
Loading 640000
Loading 670000
Loading 690000
Loading 700000
Loading 710000
Loading 740000
Loading 790000
Loading 830000
Loading 850000
Loading 930000
Loading 950000
Loading 1040000
Loading 1050000
Loading 1080000
Loading 1090000
Loading 1100000
Loading 1130000
Loading 1180000
Loading 1220000
Loading 1230000
Loading 1250000
Loading 1270000
Loading 1280000
Loading 1330000
Loading 1370000
Loading 1390000
Loading 1400000
Loading 1480000
Loading 1550000
Loading 1600000
Loading 1620000
Loading 1640000
Loading 1650000
Loading 1670000
Loading 1710000
Loading 1730000
Loading 1760000
Loading 1890000
Loading 1900000
Loading 1930000
Loading 1940000
Loading 1950000
Loading 1980000
Loading 2040000
Loading 2050000
Loading 

In [45]:
model = SentenceTransformer('bert-base-wikipedia-sections-mean-tokens', logfile="train.log")

Logs go to file train.log


In [46]:
# train_examples = all_examples[:train_dev_test_split[0]]
train_examples = all_examples[:20000]
train_data = SentencesDataset(train_examples, model, show_progress_bar=True)
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=8)



Convert dataset:   0%|          | 0/20000 [00:00<?, ?it/s]

Convert dataset:   0%|          | 34/20000 [00:00<00:59, 332.99it/s]

Convert dataset:   0%|          | 61/20000 [00:00<01:04, 307.76it/s]

Convert dataset:   0%|          | 83/20000 [00:00<01:17, 257.96it/s]

Convert dataset:   0%|          | 100/20000 [00:00<01:29, 221.18it/s]

Convert dataset:   1%|          | 131/20000 [00:00<01:23, 238.86it/s]

Convert dataset:   1%|          | 163/20000 [00:00<01:17, 257.27it/s]

Convert dataset:   1%|          | 206/20000 [00:00<01:07, 292.24it/s]

Convert dataset:   1%|          | 236/20000 [00:00<01:12, 270.84it/s]

Convert dataset:   1%|▏         | 264/20000 [00:00<01:13, 267.35it/s]

Convert dataset:   1%|▏         | 293/20000 [00:01<01:12, 271.39it/s]

Convert dataset:   2%|▏         | 321/20000 [00:01<01:20, 244.14it/s]

Convert dataset:   2%|▏         | 347/20000 [00:01<01:34, 208.91it/s]

Convert dataset:   2%|▏         | 370/20000 [00:01<01:31, 213.45it/s]

Convert dataset:  

In [47]:
# dev_examples = all_examples[train_dev_test_split[0]:train_dev_test_split[1]]

dev_examples = all_examples[train_dev_test_split[1]:train_dev_test_split[1]+1000]
dev_data = SentencesDataset(dev_examples, model, show_progress_bar=True)
dev_dataloader = DataLoader(dev_data, shuffle=False, batch_size=8)



Convert dataset:   0%|          | 0/1000 [00:00<?, ?it/s]

Convert dataset:   2%|▏         | 17/1000 [00:00<00:05, 169.95it/s]

Convert dataset:   4%|▍         | 42/1000 [00:00<00:05, 186.25it/s]

Convert dataset:   6%|▌         | 57/1000 [00:00<00:05, 171.71it/s]

Convert dataset:   8%|▊         | 81/1000 [00:00<00:04, 187.19it/s]

Convert dataset:  10%|▉         | 99/1000 [00:00<00:04, 184.78it/s]

Convert dataset:  12%|█▏        | 117/1000 [00:00<00:04, 182.79it/s]

Convert dataset:  13%|█▎        | 134/1000 [00:00<00:05, 161.71it/s]

Convert dataset:  15%|█▌        | 150/1000 [00:00<00:06, 136.45it/s]

Convert dataset:  16%|█▋        | 164/1000 [00:01<00:06, 134.98it/s]

Convert dataset:  18%|█▊        | 178/1000 [00:01<00:07, 109.88it/s]

Convert dataset:  19%|█▉        | 190/1000 [00:01<00:08, 100.32it/s]

Convert dataset:  21%|██        | 210/1000 [00:01<00:06, 116.62it/s]

Convert dataset:  22%|██▏       | 224/1000 [00:01<00:06, 117.06it/s]

Convert dataset:  24%|██▍       | 

In [48]:
train_loss = losses.CosineSimilarityLoss(model=model)
evaluator = EmbeddingSimilarityEvaluator(dev_dataloader)

In [49]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=10,
          evaluation_steps=100,
          warmup_steps=100,
          output_path="out",
          optimizer_params={'lr': 2e-5, 'eps': 1e-6, 'correct_bias': False})



Epoch:   0%|          | 0/10 [00:00<?, ?it/s]


Iteration:   0%|          | 0/2500 [00:00<?, ?it/s]

Step 0/25000: Loss val: tensor(nan, device='cuda:0', grad_fn=<MseLossBackward>)





Iteration:   0%|          | 1/2500 [00:00<18:33,  2.25it/s]

Step 1/25000: Loss val: tensor(nan, device='cuda:0', grad_fn=<MseLossBackward>)





Iteration:   0%|          | 2/2500 [00:00<18:29,  2.25it/s]

...

Step 98/25000: Loss val: tensor(nan, device='cuda:0', grad_fn=<MseLossBackward>)





Iteration:   4%|▍         | 99/2500 [00:41<16:49,  2.38it/s]

Step 99/25000: Loss val: tensor(nan, device='cuda:0', grad_fn=<MseLossBackward>)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [31]:
torch.Tensor([1, 0, 1])

tensor([1., 0., 1.])

In [35]:
import torch
torch.cosine_similarity(torch.Tensor([[1, 1, 1]]), torch.Tensor([[1, 0, 1]]), dim=1)


tensor([0.8165])